# Resistance/Support AND Candles Patterns

In [2]:
import pandas as pd
!pip install pandas_ta
import pandas_ta as ta
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import matplotlib.pyplot as plt

# df = yf.download('NVDA','2014-1-1','2024-4-30')
# # df2 = yf.download("NVDA", start="2024-05-01", end="2024-05-03",  interval = "1m")
# df.columns = [x.lower() for x in df.columns]
# df = df.drop('adj close', axis=1)
# #Check if NA values are in data
# df=df[df['volume']!=0]
# # df.reset_index(drop=True, inplace=True)
# df.isna().sum()
# df.tail()

from backtesting import Backtest, Strategy
from backtesting.test import GOOG
from backtesting.lib import crossover
# import pandas_ta as ta
import talib

df = yf.download('GOOG','2020-1-1','2024-4-30', auto_adjust=True)
#Check if NA values are in data
df=df[df['Volume']!=0]
df.isna().sum()
print(df.head)

zsh:1: command not found: pip


[*********************100%%**********************]  1 of 1 completed

<bound method NDFrame.head of                   Open        High         Low       Close    Volume
Date                                                                
2020-01-02   67.077499   68.406998   67.077499   68.368500  28132000
2020-01-03   67.392998   68.625000   67.277199   68.032997  23728000
2020-01-06   67.500000   69.824997   67.500000   69.710503  34646000
2020-01-07   69.897003   70.149498   69.518997   69.667000  30054000
2020-01-08   69.603996   70.579002   69.542000   70.216003  30560000
...                ...         ...         ...         ...       ...
2024-04-23  158.589996  160.479996  157.964996  159.919998  16115400
2024-04-24  159.089996  161.389999  158.820007  161.100006  19485700
2024-04-25  153.360001  158.279999  152.768005  157.949997  36197800
2024-04-26  175.990005  176.419998  171.399994  173.690002  56500800
2024-04-29  170.770004  171.380005  167.059998  167.899994  35914600

[1088 rows x 5 columns]>


# Support and Resistance FUNCTIONS

In [3]:
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.Low[i]>df1.Low[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.Low[i]<df1.Low[i-1]):
            return 0
    return 1

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    for i in range(l-n1+1, l+1):
        if(df1.High[i]<df1.High[i-1]):
            return 0
    for i in range(l+1,l+n2+1):
        if(df1.High[i]>df1.High[i-1]):
            return 0
    return 1

In [4]:
length = len(df)
high = list(df['High'])
low = list(df['Low'])
close = list(df['Close'])
open = list(df['Open'])
bodydiff = [0] * length

highdiff = [0] * length
lowdiff = [0] * length
ratio1 = [0] * length
ratio2 = [0] * length

def isEngulfing(l):
    """
    test if this particular candlestick is an engulfing candlestick, test its length + properties with the previous day
    or the previous candlestick and its going to return 1 if its a bearish engulfing pattern and 2 
    if its a bullish engulfing pattern.
    two candlestick patterns: when l-1 is red, and l is green, and body of l engulfs l-1, then its a bullish engulfing pattern
    
    """
    row=l
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001      

    bodydiffmin = 0.002
    if (bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]<close[row-1] and
        open[row]>close[row] and 
        (open[row]-close[row-1])>=-0e-5 and close[row]<open[row-1]): #+0e-5 -5e-5
        return 1

    elif(bodydiff[row]>bodydiffmin and bodydiff[row-1]>bodydiffmin and
        open[row-1]>close[row-1] and
        open[row]<close[row] and 
        (open[row]-close[row-1])<=+0e-5 and close[row]>open[row-1]):#-0e-5 +5e-5
        return 2
    else:
        return 0
       
def isStar(l):
    """
    test if this candlestick represents a form of rejection of the price, candles where you have a long tail with a small body
    """
    bodydiffmin = 0.0020
    row=l
    highdiff[row] = high[row]-max(open[row],close[row])
    lowdiff[row] = min(open[row],close[row])-low[row]
    bodydiff[row] = abs(open[row]-close[row])
    if bodydiff[row]<0.000001:
        bodydiff[row]=0.000001
    ratio1[row] = highdiff[row]/bodydiff[row]
    ratio2[row] = lowdiff[row]/bodydiff[row]

    if (ratio1[row]>1 and lowdiff[row]<0.2*highdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]>close[row]):
        return 1
    elif (ratio2[row]>1 and highdiff[row]<0.2*lowdiff[row] and bodydiff[row]>bodydiffmin):# and open[row]<close[row]):
        return 2
    else:
        return 0
    
def closeResistance(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.High[l]-min(levels, key=lambda x:abs(x-df.High[l])))<=lim
    c2 = abs(max(df.Open[l],df.Close[l])-min(levels, key=lambda x:abs(x-df.High[l])))<=lim
    c3 = min(df.Open[l],df.Close[l])<min(levels, key=lambda x:abs(x-df.High[l]))
    c4 = df.Low[l]<min(levels, key=lambda x:abs(x-df.High[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0
    
def closeSupport(l,levels,lim):
    if len(levels)==0:
        return 0
    c1 = abs(df.Low[l]-min(levels, key=lambda x:abs(x-df.Low[l])))<=lim
    c2 = abs(min(df.Open[l],df.Close[l])-min(levels, key=lambda x:abs(x-df.Low[l])))<=lim
    c3 = max(df.Open[l],df.Close[l])>min(levels, key=lambda x:abs(x-df.Low[l]))
    c4 = df.High[l]>min(levels, key=lambda x:abs(x-df.Low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return 1
    else:
        return 0

In [5]:
n1=2
n2=2
backCandles=30
signal = [0] * length

for row in range(backCandles, len(df)-n2):
    ss = []
    rr = []
    for subrow in range(row-backCandles+n1, row+1):
        if support(df, subrow, n1, n2):
            ss.append(df.Low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.High[subrow])
    #!!!! parameters
    if ((isEngulfing(row)==1 or isStar(row)==1) and closeResistance(row, rr, 150e-5) ):#and df.RSI[row]<30
        signal[row] = 1
    elif((isEngulfing(row)==2 or isStar(row)==2) and closeSupport(row, ss, 150e-5)):#and df.RSI[row]>70
        signal[row] = 2
    else:
        signal[row] = 0



/var/folders/hc/_w7g8hhx6y34zl23s8tbcn2w0000gn/T/ipykernel_19193/2357416705.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if(df1.Low[i]>df1.Low[i-1]):
/var/folders/hc/_w7g8hhx6y34zl23s8tbcn2w0000gn/T/ipykernel_19193/2357416705.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if(df1.High[i]<df1.High[i-1]):
/var/folders/hc/_w7g8hhx6y34zl23s8tbcn2w0000gn/T/ipykernel_19193/2357416705.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.ilo

In [6]:
df['signal'] = signal

In [7]:
df[df['signal'] == 2].count()

Open      13
High      13
Low       13
Close     13
Volume    13
signal    13
dtype: int64

In [8]:
print(df.columns)

df.columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'signal']
print(df.shape)

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'signal'], dtype='object')
(1088, 6)


In [9]:
def SIGNAL():
    return df.signal

In [20]:
#A new strategy needs to extend Strategy class and override its two abstract methods: init() and next().
#Method init() is invoked before the strategy is run. Within it, one ideally precomputes in efficient, 
#vectorized manner whatever indicators and signals the strategy depends on.
#Method next() is then iteratively called by the Backtest instance, once for each data point (data frame row), 
#simulating the incremental availability of each new full candlestick bar.

#Note, backtesting.py cannot make decisions / trades within candlesticks — any new orders are executed on the
#next candle's open (or the current candle's close if trade_on_close=True). 
#If you find yourself wishing to trade within candlesticks (e.g. daytrading), you instead need to begin 
#with more fine-grained (e.g. hourly) data.

In [11]:
from backtesting import Strategy

class MyCandlesStrat(Strategy):  
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next() 
        if self.signal1 == 2:
            self.buy()
            #TODO: use StopLoss and TakeProfit later
        elif self.signal1 == 1:
            self.position.close()

In [12]:
from backtesting import Backtest

bt = Backtest(df, MyCandlesStrat, cash=10_000, commission=.002)
stat = bt.run()
stat



Start                     2020-01-02 00:00:00
End                       2024-04-29 00:00:00
Duration                   1579 days 00:00:00
Exposure Time [%]                   48.345588
Equity Final [$]                 14216.699848
Equity Peak [$]                  14459.059696
Return [%]                          42.166998
Buy & Hold Return [%]              145.580925
Return (Ann.) [%]                    8.490297
Volatility (Ann.) [%]               24.855258
Sharpe Ratio                          0.34159
Sortino Ratio                        0.546021
Calmar Ratio                         0.225221
Max. Drawdown [%]                  -37.697577
Avg. Drawdown [%]                   -4.458364
Max. Drawdown Duration      753 days 00:00:00
Avg. Drawdown Duration       63 days 00:00:00
# Trades                                    9
Win Rate [%]                        77.777778
Best Trade [%]                      30.727278
Worst Trade [%]                    -24.771594
Avg. Trade [%]                    

In [23]:
bt.plot()

/usr/local/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.12/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/usr/local/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.12/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.ac

GridPlot(id='p1351', ...)